Create Dataset

In [1]:
#Imports
import logging
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

from medcat.cdb import CDB
from medcat.config_rel_cat import ConfigRelCAT
from medcat.rel_cat import RelCAT
from medcat.utils.relation_extraction.base_component import BaseComponent_RelationExtraction
from medcat.utils.relation_extraction.bert.model import BaseModel_RelationExtraction
from medcat.utils.relation_extraction.bert.config import BaseConfig_RelationExtraction
from medcat.utils.relation_extraction.tokenizer import BaseTokenizerWrapper_RelationExtraction

import sys, os
utils_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'utils'))
if utils_path not in sys.path:
    sys.path.insert(0, utils_path)

from utils import load_data
from relative_date_extractor import add_relative_dates

In [2]:
# Load data
original_df = load_data("../data/medcat_trainer_dataset.csv")
print(f"Loaded {len(original_df)} records")
original_df

Loaded 5 records


,doc_id,note_text,entities_json,dates_json,relative_dates_json,links_json
0,26461,Ultrasound (30nd Jun 2024): no significant fin...,"[{'id': 308244, 'value': 'history of meningiti...","[{'id': 308320, 'value': '30nd Jun 2024', 'sta...",[],"[{'date': '12nd Sep 2024', 'entity': 'pituitar..."
1,26462,Labs (27th Sep 2024): anemia. resolving Skin:...,"[{'id': 308371, 'value': 'lesions', 'cui': '52...","[{'id': 308581, 'value': '22/11/24', 'start': ...",[],"[{'date': '27th Sep 2024', 'entity': 'anemia',..."
2,26463,URGENT REVIEW (2024-10-04): cough. suspect ost...,"[{'id': 308886, 'value': 'frequent urination',...","[{'id': 308940, 'value': '2024-10-04', 'start'...",[],"[{'date': '2024-10-04', 'entity': 'cough', 'da..."
3,26464,URGENT REVIEW (13rd Feb 2025) MRI of the brain...,"[{'id': 308951, 'value': 'multiple_sclerosis',...","[{'id': 308996, 'value': '05-03-2025', 'start'...",[],"[{'date': '13rd Feb 2025', 'entity': 'visual',..."
4,26465,New pt((18/11/24)): pt presents with nausea/vo...,"[{'id': 308998, 'value': 'history of neoplasm ...","[{'id': 309070, 'value': '18/11/24', 'start': ...",[],"[{'date': '18/11/24', 'entity': 'nausea/vomiti..."


In [3]:
#Add relative dates if not already added via MedCAT trainer 
if 'relative_dates_json' not in original_df.columns:
    original_df = add_relative_dates(original_df)
    print("Added relative dates")
else:
    print("Relative dates already present, skipping extraction")

Relative dates already present, skipping extraction


In [4]:
#Insert marker function
def insert_marker(txt, start, end, tag_open, tag_close):
                    return txt[:start] + tag_open + txt[start:end] + tag_close + txt[end:]

In [5]:
# Create dataset for training
rows = []

for _, row in original_df.iterrows():
    doc_id = row["doc_id"]
    text = row["note_text"]
    
    # Parse the JSON columns - handle both string and already-parsed cases
    entities = row["entities_json"] if isinstance(row["entities_json"], list) else json.loads(row["entities_json"])
    dates = row["dates_json"] if isinstance(row["dates_json"], list) else json.loads(row["dates_json"])
    relative_dates = row["relative_dates_json"] if isinstance(row["relative_dates_json"], list) else json.loads(row["relative_dates_json"]) if "relative_dates_json" in row else []
    
    # Combine absolute and relative dates
    all_dates = dates + relative_dates
    
    # Get original links for labeling
    links = row["links_json"] if isinstance(row["links_json"], list) else json.loads(row["links_json"])
    
    # Build link pairs from validated links - match by IDs, not values
    link_pairs = {tuple(sorted([str(L["date_id"]), str(L["entity_id"])])) for L in links}
    
    # Create pairs for all date-entity combinations
    for date in all_dates:
        for entity in entities:
            # Use IDs for matching, not values
            date_id = str(date["id"])
            entity_id = str(entity["id"])
            
            # Determine label (both absolute and relative dates can have links)
            if tuple(sorted([date_id, entity_id])) in link_pairs:
                label, label_id = "LINK", 1
            else:
                label, label_id = "NO_LINK", 0
            
            # Insert markers
            s1, e1 = date.get("start"), date.get("end")
            s2, e2 = entity.get("start"), entity.get("end")
            
            if s1 is not None and s2 is not None:
                if s1 < s2:
                    marked = insert_marker(text, s2, e2, "[s2]", "[e2]")
                    marked = insert_marker(marked, s1, e1, "[s1]", "[e1]")
                else:
                    marked = insert_marker(text, s1, e1, "[s2]", "[e2]")
                    marked = insert_marker(marked, s2, e2, "[s1]", "[e1]")
            else:
                marked = text
            
            rows.append({
                "relation_token_span_ids": None,
                "ent1_ent2_start": (s1, s2),
                "ent1": date.get("value", ""),
                "ent2": entity.get("value", ""),
                "label": label,
                "label_id": label_id,
                "ent1_type": "DATE",
                "ent2_type": "ENTITY",
                "ent1_id": date_id,
                "ent2_id": entity_id,
                "ent1_cui": None,
                "ent2_cui": None,
                "doc_id": doc_id,
                "text": marked
            })

df = pd.DataFrame(rows)
print(f"Created {len(df)} date-entity pairs (including relative dates)")

Created 741 date-entity pairs (including relative dates)


In [10]:
#Inspect data
#df

In [7]:
# Per-document summary from original data
summary = []

for _, row in original_df.iterrows():
    # Parse the JSON columns - handle both string and already-parsed cases
    entities = row["entities_json"] if isinstance(row["entities_json"], list) else json.loads(row["entities_json"])
    dates = row["dates_json"] if isinstance(row["dates_json"], list) else json.loads(row["dates_json"])
    relative_dates = row["relative_dates_json"] if isinstance(row["relative_dates_json"], list) else json.loads(row["relative_dates_json"]) if "relative_dates_json" in row else []
    links = row["links_json"] if isinstance(row["links_json"], list) else json.loads(row["links_json"])
    
    summary.append({
        "doc_id": row["doc_id"],
        "n_entities": len(entities),
        "n_dates": len(dates),
        "n_relative_dates": len(relative_dates),
        "n_links": len(links),
    })

doc_level = pd.DataFrame(summary)
doc_level

,doc_id,n_entities,n_dates,n_relative_dates,n_links
0,26461,64,6,0,4
1,26462,21,7,0,11
2,26463,15,7,0,12
3,26464,11,3,0,6
4,26465,24,3,0,4


In [8]:
# Pair-based stats from the generated df
pair_stats = (
    df.groupby("doc_id")
      .agg(
          total_pairs=("label", "size"),
          link_pairs=("label", lambda s: (s == "LINK").sum()),
      )
      .reset_index()
)

pair_stats["link_pct"] = (100 * pair_stats["link_pairs"] / pair_stats["total_pairs"]).round(1)
pair_stats

,doc_id,total_pairs,link_pairs,link_pct
0,26461,384,4,1.0
1,26462,147,11,7.5
2,26463,105,12,11.4
3,26464,33,6,18.2
4,26465,72,4,5.6


In [9]:
# Merge stats with doc_level summary
doc_level = (
    doc_level.merge(pair_stats[["doc_id", "total_pairs", "link_pct"]], on="doc_id", how="left")
             .sort_values("doc_id")
             .reset_index(drop=True)
)

# Calculate additional metrics
total_pairs_overall = len(df)
link_total = (df["label"] == "LINK").sum()
link_pct_overall = 100 * (df["label"] == "LINK").mean()

print(f"total number of date-entity pairs is {total_pairs_overall}")
print(f"total number of links: {link_total}")
print(f"percentage positive class: {link_pct_overall:.1f}%")

#Look at overall doc level summary
doc_level

total number of date-entity pairs is 741
total number of links: 37
percentage positive class: 5.0%


,doc_id,n_entities,n_dates,n_relative_dates,n_links,total_pairs,link_pct
0,26461,64,6,0,4,384,1.0
1,26462,21,7,0,11,147,7.5
2,26463,15,7,0,12,105,11.4
3,26464,11,3,0,6,33,18.2
4,26465,24,3,0,4,72,5.6


In [ ]:
# Save dataset
#df.to_csv("../data/relcat_training_data.tsv", sep="\t", index=False)

In [ ]:
# Split the dataset and save

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

print(f"Train set: {len(train_df)} samples")
print(f"Test set: {len(test_df)} samples")
print("\nTrain label distribution:")
print(train_df["label"].value_counts())
print("\nTest label distribution:")
print(test_df["label"].value_counts())

# Save train and test sets
train_df.to_csv("../data/relcat_training_data.tsv", sep="\t", index=False)
test_df.to_csv("../data/relcat_test_data.tsv", sep="\t", index=False)

Config & Training

In [ ]:
#Create RelCAT config and set parameters
config = ConfigRelCAT()
config.general.log_level = logging.INFO
config.general.model_name = "bert-base-uncased" # base model that you want to use, we're going to use the HuggingFace bert-base-uncased model

#logging.basicConfig(level=logging.INFO)

In [ ]:
#Hidden size, model size and hidden layers
config.model.hidden_size= 256
config.model.model_size = 2304 # 4096 for llama

In [ ]:
# Further config
config.general.cntx_left = 15 # how many tokens to the left of the start entity we select
config.general.cntx_right = 15 # how many tokens to the right of the end entity we selecd
config.general.window_size = 300 # distance (in characters) between two entities to be considered a relation
config.train.nclasses = 2 # number of classes in your medcat export / dataset
config.train.nepochs = 3 # number of epochs to train for
config.model.freeze_layers = False # whether to freeze the layers of the base model
config.general.limit_samples_per_class = 300 # limit the number of training samples per class to this number, to avoid overfitting in unbalanced datasets
config.train.batch_size = 32 # batch size
config.train.lr = 3e-5
config.train.adam_epsilon = 1e-8
config.train.adam_weight_decay = 0.0005

In [ ]:
#Create CDB
cdb = CDB()

In [ ]:
#Create tokenizer
tokenizer = BaseTokenizerWrapper_RelationExtraction.load(tokenizer_path=config.general.model_name,
                                                       relcat_config=config)

In [ ]:
#Add special tokens
special_ent_tokens = ["[s1]", "[e1]", "[s2]", "[e2]"]
tokenizer.hf_tokenizers.add_tokens(special_ent_tokens, special_tokens=True)
tokenizer.hf_tokenizers.add_special_tokens({'pad_token': '[PAD]'}) # used in llama tokenizer

In [ ]:
#Add tokens to config
config.general.tokenizer_relation_annotation_special_tokens_tags = special_ent_tokens
config.general.annotation_schema_tag_ids = tokenizer.hf_tokenizers.convert_tokens_to_ids(special_ent_tokens)

In [ ]:
#Create RelCAT object and initialize components
# if you wish to skip the steps in section 6.1 you can pass the init_model=True arguement to intialize the components with the default ConfigRelCAT settings.
relCAT = RelCAT(cdb, config=config)

In [ ]:
# Load model configuration
model_config = BaseConfig_RelationExtraction.load(pretrained_model_name_or_path=config.general.model_name,
                                                 relcat_config=config)

In [ ]:
# Update vocab size in model config to match tokenizer
model_config.hf_model_config.vocab_size = tokenizer.get_size()

In [ ]:
# set the padding idx in the model config and relcat config, this is necesasry as it depends on what tokenizer you use
config.model.padding_idx = model_config.pad_token_id = tokenizer.get_pad_id()

In [ ]:
# Load model
model = BaseModel_RelationExtraction.load(pretrained_model_name_or_path=config.general.model_name,
                                         model_config=model_config,
                                         relcat_config=config)

In [ ]:
# Resize embeddings to match tokenizer
model.hf_model.resize_token_embeddings(len(tokenizer.hf_tokenizers)) # type: ignore

In [ ]:
# Create component
component = BaseComponent_RelationExtraction(tokenizer=tokenizer, config=config)
component.model = model
component.model_config = model_config
component.relcat_config = config
component.tokenizer = tokenizer
relCAT.component = component

In [ ]:
# Train the model using the dataset we created
relCAT.train(
    train_csv_path="../data/relcat_training_data.tsv",  
    checkpoint_path="../models/relcat_models"
)

In [ ]:
#Save model
relCAT.save(save_path="../models/relcat_models")

Inference & Testing

In [ ]:
#Load model
relCAT = RelCAT.load("../models/relcat_models")

In [ ]:
# Load test data
test_df = pd.read_csv("../data/relcat_test_data.tsv", sep="\t")
print(f"Loaded test data: {len(test_df)} samples")

In [ ]:
test_df

In [ ]:
# Run inference on all test documents using the same data source as training
all_predictions = []

test_doc_ids = test_df['doc_id'].unique()

for doc_id in test_doc_ids:
    # Find the document in medcat_df (same source as training)
    doc_row = medcat_df[medcat_df['doc_id'] == doc_id].iloc[0]
    
    # Parse the same JSON columns used in training
    entities = json.loads(doc_row["entities_json"])
    dates = json.loads(doc_row["dates_json"])
    
    # Create annotations in the same format as training
    annotations = []
    for entity in entities:
        annotations.append({
            "value": entity["value"],
            "cui": entity.get("cui"),
            "start": entity.get("start"),
            "end": entity.get("end")
        })
    for date in dates:
        annotations.append({
            "value": date["value"],
            "cui": DATE_CUI,
            "start": date.get("start"),
            "end": date.get("end")
        })
    
    try:
        # Run inference
        output_doc_with_relations = relCAT.predict_text_with_anns(
            text=doc_row["note_text"], 
            annotations=annotations
        )
        
        # Collect results - only keep date-entity pairs
        for relation in output_doc_with_relations._.relations:
            # Check if this is a date-entity pair (not entity-entity)
            if (relation['ent1_text'] in [d['value'] for d in dates] and 
                relation['ent2_text'] in [e['value'] for e in entities]) or \
               (relation['ent2_text'] in [d['value'] for d in dates] and 
                relation['ent1_text'] in [e['value'] for e in entities]):
                
                all_predictions.append({
                    'entity_label': relation['ent1_text'],
                    'date': relation['ent2_text'],
                    'confidence': relation['confidence'],
                    'doc_id': doc_id
                })
                
    except Exception as e:
        print(f"Error processing document {doc_id}: {e}")
        continue

print(f"Processed {len(test_doc_ids)} test documents")
print(f"Total predictions: {len(all_predictions)}")

In [ ]:
# Show results
print("Test Set Results:")
print(f"Total predictions: {len(all_predictions)}")

# Show first 10 predictions
print("\nFirst 10 predictions:")
for i, pred in enumerate(all_predictions[:10]):
    print(f"{i+1}. {pred['entity_label']} -> {pred['date']} (conf: {pred['confidence']:.3f}) [doc: {pred['doc_id']}]")

# Show high confidence predictions
high_conf = [p for p in all_predictions if p['confidence'] > 0.7]
print(f"\nHigh confidence predictions (>0.7): {len(high_conf)}")
for i, pred in enumerate(high_conf[:5]):  # Show first 5
    print(f"{i+1}. {pred['entity_label']} -> {pred['date']} (conf: {pred['confidence']:.3f})")

In [ ]:
# Let's debug the exact counts
print(f"Total test pairs: {len(test_df)}")
print(f"Total predictions: {len(all_predictions)}")

# Count how many test pairs were actually predicted
predicted_count = 0
for _, row in test_df.iterrows():
    found = False
    for pred in all_predictions:
        if (pred['doc_id'] == row['doc_id'] and 
            ((pred['entity_label'] == row['ent1'] and pred['date'] == row['ent2']) or
             (pred['entity_label'] == row['ent2'] and pred['date'] == row['ent1']))):
            found = True
            break
    if found:
        predicted_count += 1

print(f"Test pairs that were predicted: {predicted_count}")
print(f"Test pairs that were NOT predicted: {len(test_df) - predicted_count}")

# Also check if there are predictions for documents not in test set
test_doc_ids = set(test_df['doc_id'].unique())
pred_doc_ids = set([p['doc_id'] for p in all_predictions])
print(f"Test doc IDs: {test_doc_ids}")
print(f"Prediction doc IDs: {pred_doc_ids}")
print(f"Extra predictions (not in test): {len(pred_doc_ids - test_doc_ids)}")

In [ ]:
# Create predictions for all test pairs
all_test_predictions = []

for _, row in test_df.iterrows():
    # Check if this pair was predicted as a link
    found = False
    for pred in all_predictions:
        if (pred['doc_id'] == row['doc_id'] and 
            ((pred['entity_label'] == row['ent1'] and pred['date'] == row['ent2']) or
             (pred['entity_label'] == row['ent2'] and pred['date'] == row['ent1']))):
            all_test_predictions.append('LINK')
            found = True
            break
    
    if not found:
        all_test_predictions.append('NO_LINK')

# Now calculate metrics on all test pairs
y_true_all = test_df['label'].tolist()
y_pred_all = all_test_predictions

print(f"\nAll Test Pairs Metrics:")
print(f"Accuracy: {sum(1 for t, p in zip(y_true_all, y_pred_all) if t == p) / len(y_true_all):.3f}")
print(classification_report(y_true_all, y_pred_all, labels=['LINK', 'NO_LINK']))